In [ ]:
!pip install aiohttp nest_asyncio
!pip install pyngrok
!pip install python-dotenv
!pip install psutil
!pip install gitpython

In [ ]:
from google.colab import drive
from dotenv import load_dotenv
import os
import sys

if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')
else:
    print("Google Drive ist bereits gemountet.")

# Lade Umgebungsvariablen aus .env-Datei
target_dir = '/content/drive/MyDrive/Colab_Notebooks/MLcoworker/ColabSide'
sys.path.append(target_dir+"/prozessor_utils")

os.chdir(target_dir)
env_path = os.path.join(target_dir, '.env')
load_dotenv(env_path)


In [ ]:
import psutil
import os

def kill_port(port):
    for proc in psutil.process_iter(['pid', 'name']):
        for conn in proc.connections(kind='inet'):
            if conn.laddr.port == port:
                os.kill(proc.info['pid'], 9)  # Beende den Prozess auf dem angegebenen Port
                print(f"Prozess {proc.info['pid']} auf Port {port} wurde beendet.")

# Port 8081 freigeben, falls er belegt ist
kill_port(8081)
#letzte sinvolle benutzung vor 25.09.2024

In [ ]:
import asyncio
import os
import nest_asyncio
from flask_server import app
from ngrok_utils import start_ngrok
from pyngrok import ngrok
from werkzeug.serving import run_simple

# Installiere nest_asyncio
nest_asyncio.apply()

async def start_services():
    try:
        authtoken = os.getenv("NGROK_AUTHTOKEN")
        port = int(os.getenv("PORT", 5501))  # Ändere den Port hier auf 5501
        await start_ngrok(authtoken, port)
    except Exception as e:
        print(f"Fehler beim Starten der Dienste: {str(e)}")

async def run_flask():
    port = int(os.getenv("PORT", 5501))
    # Starte Flask im asynchronen Modus
    #app.run(port=port, debug=True)
    run_simple('0.0.0.0', port, app, use_reloader=False)
    
async def main():
    await start_services()
    # Starte Flask als asynchronen Task
    await asyncio.create_task(run_flask())

    # Halte den Event-Loop am Laufen
    while True:
        await asyncio.sleep(3600)

if __name__ == "__main__":
    # Starte den Event-Loop
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


In [ ]:
#EASY MODE

!pip install flask pyngrok python-dotenv
from flask import Flask, jsonify
from pyngrok import ngrok



app = Flask(__name__)

# Health-Check Route
@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'healthy'})

if __name__ == "__main__":
    # Hole die ngrok Authtoken und den Port aus den Umgebungsvariablen
    authtoken = os.getenv("NGROK_AUTHTOKEN")
    print(f"Der Authtoken ist: {authtoken}")
    port = int(os.getenv("PORT", 5000))
    print(f"Der Port ist: {port}")

    # Starte ngrok und öffne den Tunnel
    ngrok.set_auth_token(authtoken)
    public_url = ngrok.connect(port).public_url 
    print(f"ngrok-Tunnel geöffnet unter {public_url}")
    print("Test mit:")
    print(f"curl {public_url}/health")

    # Starte den Flask-Server auf dem angegebenen Port
    app.run(port=port)
